In [1]:
import torch
import pandas as pd
import numpy as np
import pickle
import plotly.graph_objects as go
import sklearn
from sklearn.cluster import DBSCAN
import os
import sys
ROOT = "./"
sys.path.append(ROOT + "lib")

from plotly_style import update_layout
from lightsource import LightSource
from helpers import to_inference_input
# with open(ROOT + "models/model.pt", "rb") as f:
#   model = torch.load(f)
# state = torch.load(ROOT + "models/model.pt")
TEST = True
BATCHSIZE = 10

In [ ]:
object_key = {}
row_key = {}
raw = pd.DataFrame()

if TEST:
  with open(ROOT + "cached_data/test_buckets.pkl", "rb") as f:
    test_buckets = pickle.load(f)

  for kind in test_buckets:
    for obj in test_buckets[kind]:
      tbl = obj.get_pandas()
      ra = tbl["ra"].to_numpy()
      dec = tbl["dec"].to_numpy()
      center = (np.mean(ra), np.mean(dec))
      object_key[center] = kind

      start = len(raw)
      end = start + len(tbl) - 1
      raw = pd.concat((raw, tbl), ignore_index=True)
      for i in range(start,end):
        row_key[i] = kind

  raw.insert(len(raw.columns), "idx", list(range(len(raw))), True) #indexing
else:
  pass
raw

Clustering and collecting objects

In [7]:
clstr_tbl = raw[50000:58000][["ra", "dec"]]
dbscan = DBSCAN(eps=0.7, min_samples=10).fit(clstr_tbl)
labels = dbscan.labels_

objects = {}

for i, label in enumerate(labels):
  if label != -1:
    if label not in objects:
      objects[label] = raw.iloc[[i]]
    else:
      objects[label] = pd.concat((objects[label], raw.iloc[[i]]))

In [ ]:
objects = [LightSource(tbl) for tbl in objects]
feed = [to_inference_input(obj.to_tensor()) for obj in objects]

for obj in feed:
    
    